## Local Function Calling
The **LocalCodeTool** ant demonstrates the use of .Net functions as tools! You don't have to use a Web API. 

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.9.1"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.9.1

## Load the assembly containing the functions
AntLoader.LoadAssembly ensures the assembly is loaded and findable by AntRunnerLib

**Don't load any assemblies you don't fully trust!**

In [2]:
AntLoader.LoadAssembly(@".\AssistantDefinitions\LocalCodeTool\TestAssistant.dll");

## Ensure the LocalCodeTool assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\LocalCodeTool" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"LocalCodeTool" contains the following files:
```
│   instructions.md - The instructions
│   LocalFunction.cs - The code that defines the two functions for your reference
│   manifest.json - The base assistant definition
│   TestAssistant.dll - A .NET 8 library we which contains the local functions in executable form
│
└───OpenAPI
        tool.localhost.json - a swagger file where the server url is "url": "tool://localhost" and Paths are names of static methods in any loaded assembly
```

### Explanation
`AssistantUtility.Create` acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the tool.localhost swagger which follows an Open API schema into tool definitions for the assistant.

In [3]:
var assistantId = await AssistantUtility.Create("LocalCodeTool", config);
Console.WriteLine(assistantId)

asst_umiFIb2ccjyIrNinxBOWPMM6


## Run the Assistant

`output.Dialog` shows the conversation.

In [4]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "LocalCodeTool",
    Instructions = "Use both of your tools to answer these two questions: What is the date and what is eleven plus negative 42?",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: Use both of your tools to answer these two questions: What is the date and what is eleven plus negative 42?

Assistant: I called the tool named GetDateAndTime with these arguments:
```
{}
```
and got this result:
```
"Wednesday, October 2, 2024 9:58:13 PM"
```

I called the tool named AddNumbers with these arguments:
```
{"a": 11, "b": -42}
```
and got this result:
```
-31
```

The current date is **Wednesday, October 2, 2024**. The result of eleven plus negative forty-two is **-31**.


In [5]:
output.LastMessage.DisplayAs("text/markdown");

The current date is **Wednesday, October 2, 2024**. The result of eleven plus negative forty-two is **-31**.

## Clean Up

In [6]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "LocalCodeTool");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find MsGraphUserProfile");
}

Deleted assistant
